# Análise de texto de fontes desestruturadas e Web

## Aula 13

Nesta aula continuaremos e explorar **Análise de Tópicos** - **topic modeling** para verificar padrões (tópicos) que emergem de forma natural nos dados.

A biblioteca utilizada será a **Gensim**. Para conhecer mais sobre ela, acesse https://radimrehurek.com/gensim/index.html e https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html

Além disso, utilizaremos o **spacy** para fazer o reconhecimento de entidades nomeadas.

## Baixar os dados no Colab

Para baixar os dados no colab, utilize:

In [ ]:
!wget https://atd-insper.s3.us-east-2.amazonaws.com/aula13/noticias_base.csv

## Instalando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [ ]:
# Remova o comentário caso o import falhe
# !pip install gensim

In [ ]:
# Remova o comentário caso o import falhe
# !pip install nltk

In [ ]:
# Para reconhecimento de entidades nomeadas
!pip install -U pip
!pip install -U spacy
!python -m spacy download pt

In [ ]:
# Para NMF
!pip install -U scikit-learn

## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [ ]:
# para trabalhar com diretórios / sistema operacional
import os

# para trabalhar com expressões regulares
import re

# utilizada para nos indicar o caminho do executável do Python
import sys

# para pandas DataFrame
import pandas as pd

# topic Modeling e NER
import gensim
from gensim import corpora
from gensim import models
from gensim.models import Nmf
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import spacy
from spacy import displacy

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Exemplo NMF
from sklearn.decomposition import NMF
import numpy as np

Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

Você pode conferir de onde está executando o Python e qual a versão

In [ ]:
print('Executável:')
print(sys.executable)

print('\nVersão do Python:')
print(sys.version)

Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [ ]:
print('O seu notebook está na pasta:')
print(os.getcwd())

Versão das bibliotecas

In [ ]:
print('gensim {}'.format(gensim.__version__))
print('nltk {}'.format(nltk.__version__))

# Exemplo

Vamos aprender como realizar modelagem de tópicos com o Python. Para isso, vamos criar um DataFrame de exemplo

In [ ]:
dfex = pd.DataFrame({'Texto': ['ações de mineradoras começam a despertar interesse dos investidores',
                             'cresce investimento nas mineradoras, entre elas a vale devido aos dividendos',
                             'ações de mineradoras sofrem com o lockdown na china devido ao covid',
                             'china cresce menos que o esperado devido ao mercado de tecnologia',
                             'sobram vagas e faltam candidatos no mercado de tecnologia',
                             'a covid continuará por muito tempo presente, precisamos nos vacinar',
                             'o mercado de tecnologia continua aquecido, com crescimento projetado de 13%']})
dfex

Vamos considerar que cada linha deste DataFrame representa um **documento**. Ou seja, temos seis documentos nesta base de dados.

Perceba que o DataFrame possui apenas uma coluna de textos, ou seja, não nenhuma variável que indique categorias. Apesar disso, podemos facilmente perceber que alguns documentos falam de assuntos específicos.

Apesar de ser fácil para um humano obter esta percepção, sabemos que isto é impraticável na escala de dados produzida pelas organizações. Então, vamos precisar de técnicas de Data Science e do Python!

# Pré-processamento

Utilizaremos diversas técnicas aprendidas durante o curso para limpar e transformar os textos que queremos analisar

Vamos converter o texto para **minúsculo**

In [ ]:
dfex['Texto'] = dfex['Texto'].str.lower()

dfex['Texto'] = dfex['Texto'].apply(remove_stopwords, stopwords='portuguese')

dfex

In [ ]:
stopwords.words('portuguese')

E separar o texto em uma **lista de palavras** (tokenização)

In [ ]:
token_texto = dfex['Texto'].apply(word_tokenize)
token_texto

Em seguida, vamos criar um **corpus** textual, que conterá uma representação dos termos ou palavras presentes em cada documento. Aqui, teremos uma representação **Bag of Words**, semelhante ao visto nas duas últimas aulas.

In [ ]:
dic = corpora.Dictionary(token_texto)
corpus = [dic.doc2bow(lista) for lista in token_texto]
corpus

In [ ]:
# Conferindo a vetorização de uma frase qualquer
dic.doc2bow("vale hoje: a mineradora vale está em alta com os investidores".split())

In [ ]:
# Entendendo os IDs
dic.token2id

## Non-Negative Matrix Factorization

O NMF (*Non-Negative Matrix Factorization*) irá decompor uma matriz de documentos e palavras (a matriz da vetorização *bag-of-words*) em duas outras matrizes: uma de palavras e tópicos e outra de tópicos e documentos.

Veja uma imagem ilustrativa:

<img src="https://atd-insper.s3.us-east-2.amazonaws.com/aula13/img/nmf.png">

Assim, podemos saber com quais tópicos cada documento está associado e quais palavras compõem cada tópico.

Obs: o produto das matrizes de variáveis e componentes será uma aproximação da matriz original de documentos e palavras (BoW).

Veja um exemplo, onde a matriz `X` é decomposta nas matrizes `W` e `H`, considerando `4` componentes:

Dica: geralmente é utilizado um número de componentes menor que o tamanho da matriz.

In [ ]:
X = np.array([[1, 2], 
              [3, 4]])

model = NMF(n_components=4, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_

print(f'Matriz W:\n{W}\n')
print(f'Matriz H:\n{H}')

Conferindo:

In [ ]:
np.dot(W, H)

### NMF utilizando `gensim`

Com o uso da biblioteca `gensim`, vamos criar uma instância de `Nmf` parametrizando a quantidade de tópicos e o dicionário para tradução das palavras.

Veja mais detalhes da documentação em https://radimrehurek.com/gensim/models/nmf.html

Vamos 

In [ ]:
nmf = Nmf(corpus, num_topics=2, id2word=dic)

lista_topico = nmf.print_topics(num_words=3)

for topico in lista_topico:
    print(topico)

Teste com quantidade diferentes de tópicos e analise os resultados

In [ ]:
nmf = Nmf(corpus, num_topics=3, id2word=dic)

lista_topico = nmf.print_topics(num_words=3)

for topico in lista_topico:
    print(topico)

## Plot dos tópicos encontrados

É conveninente ter a possibilidade de visualizar graficamente os tópicos encontrados.

Para isso, utilizaremos uma biblioteca chamada **plotly**.

In [ ]:
nmf_top = nmf.show_topics(formatted=False)

topico = nmf_top[0][1]
topico.sort(key=lambda x: x[1], reverse=True)

palavra = list(zip(*topico))[0]
score = list(zip(*topico))[1]

px.bar(x=palavra, y=score, labels={'x': 'Palavra', 'y': 'Score'}, title='Tópico 0')

In [ ]:
topico = nmf_top[1][1]
topico.sort(key=lambda x: x[1], reverse=True)

palavra = list(zip(*topico))[0]
score = list(zip(*topico))[1]

px.bar(x=palavra, y=score, labels={'x': 'Palavra', 'y': 'Score'}, title='Tópico 1')

In [ ]:
nmf_top = nmf.show_topics(formatted=False)

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('Tópico 1', 'Tópico 2'))

for i in range(2):
    topico = nmf_top[i][1]
    topico.sort(key=lambda x: x[1], reverse=True)

    palavra = list(zip(*topico))[0]
    score = list(zip(*topico))[1]
    
    fig.add_trace(go.Bar(x=palavra, y=score,
                         marker=dict(color=score, coloraxis="coloraxis")), 1, i+1)
    
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.show()

## Composição dos documentos

Um documento pode ter frases ou partes que falam de diversos assuntos, assim, será composto por um ou mais tópicos. Vamos analisar com quais tópicos um documento mais se relaciona

In [ ]:
nmf_res = nmf[corpus]

for doc, as_text in zip(nmf_res, dfex['Texto']):
    print(doc, as_text)

### Testando para novas frases

In [ ]:
doc = ['ações da ptr4 desabam devido a interferências do governo',
       'empresas que não investirem em dados e tecnologia terão um futuro duro pela frente',
       'mineradoras também precisam investir em tecnologia, elas sofrem mas investem']

token_frase = [word_tokenize(frase) for frase in doc]

corpus_ex = [dic.doc2bow(lista) for lista in token_frase]
nmf_res = nmf[corpus_ex]

for doc, as_text in zip(nmf_res, doc):
    print(doc, as_text)

## NER - Named Entity Recognition

É uma necessidade comum entender sobre o que certo documento está falando. Nas aulas anteriores, vimos que podemos utilizar expressões regulares para identificar certas palavras ou padrões de nosso interesse. Assim, poderíamos por exemplo filtrar todos os ducumentos que mencionam `PTR4` ou demais palavra associadas a Petrobras.

Com o uso de técnicas de Machine Learning supervisionadas, conseguimos treinar um modelo que consegue predizer a categoria de uma notícia. Além disso, com o uso de *topic modeling*, podemos identificar padrões que emergem naturalmente dos textos, podendo interpretar estes padrões uma vez que sabemos quais palavras compõem o tópico e com quais tópicos o documento mais se associa.

Agora, iremos apresentar uma alternativa: detecção de entidades nomeadas. Uma entidade nomeada é basicamente um objeto que possui identificação adequada e pode ser denotado com um nome próprio. Entidades Nomeadas podem ser um lugar, pessoa, organização, objeto, entidade geográfica, etc.

Do ponto de vista técnico, podemos treinar nosso próprio modelo para NER, ou utilizar modelos pré-treinados. Estes modelos já incorporam conhecimento sobre entidades comuns em determinada lingua (português, inglês) ou cenário (economia, tecnologia).

Vamos a um exemplo. Primeiro, abrimos o modelo

In [ ]:
model = spacy.load('pt_core_news_sm')

E criamos um texto qualquer com uma notícia

In [ ]:
txt = 'Sou de São Paulo, no Brasil e nasci em 1988. As ações PTR4 não param de cair na Bovespa.'

Vamos transformar a string em um documento `spacy`!

In [ ]:
doc = model(txt)

e imprimir as entidades nomeadas identificadas no texto

In [ ]:
print([(entity, entity.label_) for entity in doc.ents])

Veja um outro exemplo

In [ ]:
noticia = 'Depois de anos de fracassos e adiamentos, a companhia aeronáutica americana Boeing tentará voltar à concorrência com a SpaceX para servir de “táxi” espacial para a Nasa, na Flórida'
doc_noticia = model(noticia)
print([(entity, entity.label_) for entity in doc_noticia.ents])

Podemos pedir ao `spacy` que explique o que quer dizer as siglas utilizadas para identificar as entidades

In [ ]:
spacy.explain('LOC')

Veja uma forma diferente de visualizar as entidades, com o uso de um laço `for`

In [ ]:
for word in doc_noticia.ents:
    print(word.text,word.label_)

Podemos também exibir graficamente as entidades identificadas

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
displacy.render(doc_noticia, style="ent", jupyter=True)

Um exemplo mais completo:

In [ ]:
descricao = '''SÃO PAULO (Reuters) – O principal índice da bolsa brasileira subiu nesta quinta-feira, impulsionado por ações de siderúrgicas e mineradoras, destoando do recuo em Wall Street, que teve outro dia volátil.

Vale cresceu na esteira dos preços do minério de ferro e CSN disparou com anúncio de recompra de ações. A Eletrobras também foi destaque positivo, após aval do Tribunal de Contas da União (TCU) para privatização. WEG e Hapvida foram destaques de queda.


O Ibovespa subiu 0,71%, a 107.005,22 pontos. O volume financeiro da sessão foi de 24,7 bilhões de reais.

“O mercado ficou bem pressionado na véspera e teve um pouco de recuperação em função da alta das commodities”, disse Luiz Roberto Monteiro, operador da mesa institucional da Renascença. Ele também citou recentes elevações das projeções de crescimento econômico para o Brasil neste ano.

A XP dobrou a estimativa e agora espera expansão de 1,6% do Produto Interno Bruto (PIB)do país em 2022, embora tenha elevado também a expectativa de inflação para até 2023. E o ministério da Economia manteve sua projeção para o PIB em 1,5%, mas espera uma inflação mais alta. '''

print(descricao)

In [ ]:
doc = model(descricao)

print([(entity, entity.label_) for entity in doc.ents])

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

# Exercícios

**Exercício 1** Crie uma função de pré-processamento de textos. A função recebe uma string e:
- Remove caracteres numéricos
- Remove acentuação
- Transforma tudo para minúsculo
- Remove stopwords
- Retorna a lista de palavras contidas na string

Você pode reaproveitar a função feita na aula passada!

In [ ]:
def preproc_texto(txt):
    
    return txt_limpo

# Resultado esperado ['hey', 'vamos', 'aprender', 'nlp', 'estudo', 'vezes', 'semana']
preproc_texto('Hey, vamos aprender NLP??? Eu estudo 3 vezes na semana.')

**Exercício 2** Crie uma função que recebe um PandasSeries onde cada linha representa um texto.
Sua função deve aplicar a função de `preproc_texto` em cada linha, devolvendo um PandasSeries onde cada linha contem uma lista de tokens (conforme exercício anterior)

**Exercício 3** Aplique **NMF** à base `noticias_base.csv` para identificar os tópicos que emergem a partir da análise do **`Título`** das notícias. Você mesmo pode atribuir um número de tópicos e quantidade de palavras na análise (teste diferentes valores).

OBS: Faça todo o pré-processamento, tokenização, aplicação do `NMF`, etc. conforme visto no handout.

**Exercício 4** Crie uma função que recebe um texto e devolve as entidades nomeadas.

**Exercício 5** Crie uma função que recebe um PandasSeries onde cada linha representa um texto. Reconheça e devolva as entidades nomeadas com o uso da função feita no exercício anterior (aplique em cada linha).

Teste com a base `noticias_1.xlsx` considerando a coluna **`Descrição`**.